<a href="https://colab.research.google.com/github/Jyotirmoyee18/TTS_FineTuning_GenAI_Assignment/blob/main/Fast_Inference_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/585M [00:00<?, ?B/s]

In [7]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("DeepDiveDev/speecht5_finetuned_English")
model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/speecht5_finetuned_English")

preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")

preprocessor_config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

spm_char.model:   0%|          | 0.00/238k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/578M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# Load your pretrained or fine-tuned model
processor = SpeechT5Processor.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")

# Set the model to evaluation mode (important for quantization)
model.eval()

# Apply dynamic quantization (weights and activations in int8)
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the quantized model
torch.save(quantized_model.state_dict(), "quantized_speecht5_bengali.pth")


In [11]:
import torch
import torch.nn.utils.prune as prune

# Define a function to prune linear layers
def prune_model(model, amount=0.3):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name='weight', amount=amount)
    return model

# Apply pruning to your model
pruned_model = prune_model(model, amount=0.3)  # 30% pruning

# Save the pruned model
torch.save(pruned_model.state_dict(), "pruned_speecht5_bengali.pth")

# Load the pruned model
pruned_model = torch.load("pruned_speecht5_bengali.pth", map_location=torch.device('cpu'))

<ipython-input-11-4fad510b02d9>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pruned_model = torch.load("pruned_speecht5_bengali.pth", map_location=torch.device('cpu'))


In [19]:
!pip install datasets  # Install the 'datasets' package
from datasets import load_dataset

# Load speaker embeddings using the available 'validation' split
speaker_embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# Select a speaker embedding
speaker_embeddings = torch.tensor(speaker_embeddings_dataset[0]["xvector"]).unsqueeze(0)

In [21]:
import time
import torch
from datasets import load_dataset
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# Load the fine-tuned model and processor
processor = SpeechT5Processor.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/Bengali_finetuned_speecht5_tts")

# Set the model to evaluation mode
model.eval()

# Example text for inference
text = "আপনি কেমন আছেন?"  # Bengali text

# Tokenize text input (convert to input IDs)
inputs = processor(text=text, return_tensors="pt")

# Load speaker embeddings using the available 'validation' split
speaker_embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")  # Changed split to 'validation'
speaker_embeddings = torch.tensor(speaker_embeddings_dataset[0]["xvector"]).unsqueeze(0)

# Measure inference time
start_time = time.time()
with torch.no_grad():
    spectrogram = model.generate(
        inputs["input_ids"],
        speaker_embeddings=speaker_embeddings
    )
end_time = time.time()

# Calculate inference time
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")

Inference Time: 3.9312 seconds


In [23]:
import time
import torch
from datasets import load_dataset
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

# Load the fine-tuned English model and processor
processor = SpeechT5Processor.from_pretrained("DeepDiveDev/speecht5_finetuned_English")
model = SpeechT5ForTextToSpeech.from_pretrained("DeepDiveDev/speecht5_finetuned_English")

# Set the model to evaluation mode
model.eval()

# Example text for inference (English text)
text = "How are you doing today?"

# Tokenize text input (convert to input IDs)
inputs = processor(text=text, return_tensors="pt")

# Load speaker embeddings
# Changed 'train' to 'validation' to match the available split in the dataset
speaker_embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(speaker_embeddings_dataset[0]["xvector"]).unsqueeze(0)

# Measure inference time
start_time = time.time()
with torch.no_grad():
    spectrogram = model.generate(
        inputs["input_ids"],
        speaker_embeddings=speaker_embeddings
    )
end_time = time.time()

# Calculate inference time
inference_time = end_time - start_time
print(f"Inference Time: {inference_time:.4f} seconds")

Inference Time: 6.5786 seconds
